In [1]:
## Import packages
import numpy as np
import pandas as pd
import copy
# from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer


import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

In [2]:
train = pd.read_csv('train.csv/train.csv')
train.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
# np.array_equal(train.id, train.id.astype(int))

In [4]:
train.shape

(20800, 5)

In [5]:
train.dtypes

id         int64
title     object
author    object
text      object
label      int64
dtype: object

In [6]:
test = pd.read_csv('test.csv/test.csv')
test.shape

(5200, 4)

In [7]:
submit = pd.read_csv('submit.csv')

In [8]:
submit

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1
...,...,...
5195,25995,0
5196,25996,1
5197,25997,0
5198,25998,1


In [9]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [10]:
test.dtypes

id         int64
title     object
author    object
text      object
dtype: object

In [11]:
test.isna().sum().sort_values(ascending = False)

author    503
title     122
text        7
id          0
dtype: int64

In [12]:
# make a copy of the dataframe in case 
train_original = copy.deepcopy(train)

In [13]:
train["label"].value_counts()

#see distribution of 1 and 0 labels 

1    10413
0    10387
Name: label, dtype: int64

In [14]:
train.isna().sum().sort_values(ascending = False)

author    1957
title      558
text        39
label        0
id           0
dtype: int64

# Text pre processing 

In [15]:
# basic pre-processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

from wordcloud import WordCloud

In [16]:
def nans(train): 
    return train[train.isnull().any(axis=1)]
nans(train)

,id,title,author,text,label
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
20,20,News: Hope For The GOP: A Nude Paul Ryan Has J...,NaN,Email \nSince Donald Trump entered the electio...,1
23,23,Massachusetts Cop’s Wife Busted for Pinning Fa...,NaN,Massachusetts Cop’s Wife Busted for Pinning Fa...,1
31,31,Israel is Becoming Pivotal to China’s Mid-East...,NaN,Country: Israel While China is silently playin...,1
...,...,...,...,...,...
20745,20745,Thomas Frank Explores Whether Hillary Clinton ...,NaN,Thomas Frank Explores Whether Hillary Clinton ...,1
20768,20768,Osama bin Laden’s older brother rents out luxu...,NaN,Osama bin Laden’s older brother rents out luxu...,1
20771,20771,NaN,Letsbereal,DYN's Statement on Last Week's Botnet Attack h...,1
20772,20772,NaN,beersession,Kinda reminds me of when Carter gave away the ...,1


In [17]:
train = train.dropna(subset=['title', 'text'])

test = test.dropna(subset=['title', 'text'])

In [18]:
train.shape

(20203, 5)

In [19]:
test.shape

(5071, 4)

In [20]:
# plt.subplots(figsize=(25,15))
# wordcloud = WordCloud(
#                           background_color='white',
#                           width=1920,
#                           height=1080
#                          ).generate(" ".join(train.title))
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.savefig('b4_clean_title.png')
# plt.show()

# #Name

In [21]:
# plt.subplots(figsize=(25,15))
# wordcloud = WordCloud(
#                           background_color='white',
#                           width=1920,
#                           height=1080
#                          ).generate(" ".join(train.text))
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.savefig('b4_clean_text.png')
# plt.show()

# #Name

In [22]:
# basic pre-processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()


#remove stop words 
#lemmetization instead of stemmer 

def preprocess(review):
    review = " ".join([stemmer.stem(w.lower()) for w in word_tokenize(re.sub('[^a-zA-Z]+', ' ', review.replace("<br />", ""))) if not w in stop_words])
    return review


#cleaning all text from both test and train set 
train['text_clean'] = train.apply(lambda x: preprocess(x['text']), axis=1)
test['text_clean'] = test.apply(lambda x: preprocess(x['text']), axis=1) 

In [29]:
test.head(5)

,id,title,author,text,text_clean
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",palo alto calif after year scorn polit process...
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,russian warship readi strike terrorist near al...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,video nodapl nativ american leader vow stay al...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",if first succeed tri differ sport tim tebow he...
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,min ago view comment like for first time histo...


In [23]:
# # basic pre-processing
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# import re

# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()


# #remove stop words 
# #lemmetization instead of stemmer 

# def preprocess(review):
#     review = " ".join([lemmatizer.lemmatize(w.lower()) for w in word_tokenize(re.sub('[^a-zA-Z]+', ' ', review.replace("<br />", ""))) if not w in stop_words])
#     return review


# #cleaning all text from both test and train set 
# train['text_clean'] = train.apply(lambda x: preprocess(x['text']), axis=1)
# test['text_clean'] = test.apply(lambda x: preprocess(x['text']), axis=1) 

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# split dataset to training and test dataset
from sklearn.feature_extraction.text import CountVectorizer #one hot 
from sklearn.model_selection import train_test_split


data_train, data_val, y_train, y_val = train_test_split(train['text_clean'], train['label'], 
                                                          test_size = 0.25, random_state = 2020, stratify = train['label'])

vectorizer = CountVectorizer()
train_counts = vectorizer.fit_transform(data_train)
val_counts = vectorizer.transform(data_val)


# vectorizer = TfidfVectorizer()
# train_vectors = vectorizer.fit_transform(train['text_clean'])
# test_vectors = vectorizer.transform(test['text_clean'])

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from datetime import datetime

start=datetime.now()
lr = LogisticRegression(max_iter = 6000)

lr.fit(train_counts, y_train)
lr_pred = lr.predict(val_counts)
print('Logistic Regression')
print('Accuracy:')
print(lr.score(val_counts, y_val))
print('Confusion Matrix:')
print(confusion_matrix(lr_pred, y_val))
print('Time taken:')
print(datetime.now()-start)

Logistic Regression
Accuracy:
0.9475351415561275
Confusion Matrix:
[[2452  120]
 [ 145 2334]]
Time taken:
0:00:13.334347


In [26]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lr, train_counts, y_train, cv=10)
print('Cross-Validation Accuracy Scores', scores)

Cross-Validation Accuracy Scores [0.93865435 0.94722955 0.95577558 0.94785479 0.96105611 0.94257426
 0.95379538 0.95313531 0.94653465 0.94785479]


In [27]:
#Prediction to test set